In [2]:
class Board:
    def __init__(self, size, player_turn = 'X'):
        self.size = size
        self.board = [[0 for _ in range(size)] for _ in range(size)]
        self.turn = player_turn

    def get_tile(self, row, col):
        return self.board[row][col]

    # Check if the game is won
    def get_winner(self):
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                return self.board[i][0]
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != 0:
                return self.board[0][i]
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            return self.board[0][0]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != 0:
            return self.board[0][2]
        return 0
    
    def is_full(self):
        for row in self.board:
            for cell in row:
                if cell == 0:
                    return False
        return True
    
    def game_is_over(self):
        return self.is_full() or self.get_winner() != 0
    
    def switch_turn(self):
        if self.turn == 'X':
            return 'O'
        else:
            return 'X'
    

    def make_move(self, row, col):
        if not self.game_is_over():
            if self.board[row][col] == 0:
                self.board[row][col] = self.turn
                self.turn = self.switch_turn()
                return True
            else: 
                return False
        else:
            print(self.print_board())
            raise ValueError("Game is over")
        
    def get_possible_moves(self):
        moves = []
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] == 0:
                    moves.append((row, col))
        return moves
    
    def print_board(self):
        for i in range(self.size):
            for j in range(self.size):
                print(self.board[i][j], end=' ')
            print()
        return ''
    
    def create_copy(self):
        new_board = Board(self.size)
        new_board.board = [row[:] for row in self.board]
        new_board.turn = self.turn
        return new_board
    
    def get_board(self):
        return self.board
    
    def move_block_opponent(self, i, j):
        if not self.board[i][j] == 0:
            raise ValueError("Invalid move")
        self.board[i][j] = 'X' if self.turn == 'O' else 'O'
        if self.get_winner() != 0:
            self.board[i][j] = 0
            return True
        else:
            self.board[i][j] = 0
            return False


    
    

In [3]:
# Test to see if Board class works as expected
tic_tac_toe = Board(3)
tic_tac_toe.print_board()
print()
tic_tac_toe.make_move(0, 0)
tic_tac_toe.print_board()
print()
tic_tac_toe.make_move(2, 1)
tic_tac_toe.print_board()
print()
tic_tac_toe.make_move(1, 0)
tic_tac_toe.print_board()
print()
tic_tac_toe.make_move(2, 2)
tic_tac_toe.print_board()
print()
tic_tac_toe.make_move(2, 0)
tic_tac_toe.print_board()
print()
print("The winner is", tic_tac_toe.get_winner())

0 0 0 
0 0 0 
0 0 0 

X 0 0 
0 0 0 
0 0 0 

X 0 0 
0 0 0 
0 O 0 

X 0 0 
X 0 0 
0 O 0 

X 0 0 
X 0 0 
0 O O 

X 0 0 
X 0 0 
X O O 

The winner is X


In [4]:
import math
class Node:
    def __init__(self, board):
        self.state = board
        self.children = []
        self.n = 0
        self.Q = 0
        self.parent = None
    
    def add_child(self, child):
        self.children.append(child)
    
    def get_children(self):
        return self.children
    
    def get_state(self):
        return self.state
    
    def set_parent(self, parent):
        self.parent = parent

    def get_parent(self):
        return self.parent
    
    def has_children(self):
        return len(self.children) > 0

    def get_total_N1(self):
        total_N = self.n  # Start with the current node's N value
        current_parent = self.parent
        while current_parent is not None:
            total_N = current_parent.n  # Add the N value of the parent node
            current_parent = current_parent.parent  # Move to the next parent node
            if current_parent is None:
                break
        return total_N
    
    def get_total_N(self):
        if self.parent is None:
            return self.n  # Base case: if there's no parent, return the current node's N value
        else:
            return self.parent.get_total_N()  # Recursive case: add current node's N to parent's total

    
    def get_UCB(self):
        total_N = self.get_total_N()
        if self.n == 0:
            # Inf means it has not been visited yet
            return float('inf')
        if total_N < 0:
            print('the total N is', total_N, 'for node', self.state.get_board())
        return self.Q/self.n + 2 * (math.log(total_N)/ (self.n)) ** 0.5
    
    def is_fully_expanded(self):
        return len(self.children) == len(self.state.get_possible_moves())
    
    def is_terminal(self):
        return self.state.game_is_over()

In [5]:
import random
from math import inf
import numpy as np

class MachinePlayer:
    def __init__(self, player, max_depth):
        self.player = player
        self.max_depth = max_depth

    def get_move_made(self, board1, board2):
        for row in range(len(board1.get_board())):
            for col in range(len(board1.get_board())):
                if board1.get_tile(row, col) != board2.get_tile(row, col):
                    return (row, col)
        return None
    
    def get_best_move(self, board):
        return self.monte_carlo_tree_search(board.create_copy())

    
    def selection(self, node):
        if node.is_terminal():
            return node
        if not node.has_children():
            return node
                
        children = node.get_children()
    
        # Recursively call selection on the child with max UCB
        max_ucb_child = max(children, key=lambda n: n.get_UCB())
        
        return self.selection(max_ucb_child)


    def get_best_child(self, node):
        children = node.get_children()
      #  print('children', children)
        best_child = None
        best_score = -inf
        for child in children:
            if child.get_UCB() >= best_score: #and child.get_state().game_is_over() == False:
                best_child = child
                best_score = child.get_UCB()
        return best_child
    
    def expansion(self, node):
        if node.is_terminal():
            return node
        board = node.state.create_copy()
        possible_moves = board.get_possible_moves()
        for move in possible_moves:
            new_board = board.create_copy()
            new_board.make_move(move[0], move[1])
            new_node = Node(new_board)
            new_node.set_parent(node)
            node.add_child(new_node)
        return node
    
    def simulation(self, node):
        if len(node.get_children()) == 0:
            return node.get_state().get_winner()
        else:
            # Roll out policy
            # If win do
            # If other win, block
            # If no win, random move
            to_sim_from = node.get_state().create_copy()
            while not to_sim_from.game_is_over():
                possible_moves = to_sim_from.get_possible_moves()
                has_made_move = False
                for possible_move in possible_moves:
                    block_move = None
                    board_copy = to_sim_from.create_copy()
                    if board_copy.move_block_opponent(possible_move[0], possible_move[1]):
                        block_move = possible_move

                    board_copy.make_move(possible_move[0], possible_move[1])
                    if board_copy.get_winner() == self.player:
                        to_sim_from.make_move(possible_move[0], possible_move[1])
                        return to_sim_from.get_winner()       

                if block_move is not None:
                    to_sim_from.make_move(block_move[0], block_move[1])
                    has_made_move = True
                    
                if not has_made_move:
                    move = random.choice(possible_moves)
                    to_sim_from.make_move(move[0], move[1])

        return to_sim_from.get_winner()
    
    def get_other_player(self):
        if self.player == 'X':
            return 'O'
        else:
            return 'X'
            
    def backpropagate(self, node, result):
        current_node = node
        while current_node is not None:
            current_node.n += 1
            if result == self.player:
                current_node.Q += 1
            elif result == 0:
                current_node.Q += 0
            else:
                current_node.Q -= 1
            current_node = current_node.parent
  


    def immediate_win_or_block(self, node):
        possible_moves = node.get_state().get_possible_moves()
        blocking_move = None

        for move in possible_moves:
            board_copy = node.get_state().create_copy()
            if board_copy.move_block_opponent(move[0], move[1]):
                blocking_move = move

            board_copy.make_move(move[0], move[1])
            if board_copy.get_winner() == self.player:
                return move
            
            
        return blocking_move
    
    def monte_carlo_tree_search(self, board):
        root = Node(board)

        if self.immediate_win_or_block(root) is not None:
            return self.immediate_win_or_block(root)
        
        for i in range(self.max_depth):
            leaf = self.selection(root)
            leaf = self.expansion(leaf)
            simulation_result = self.simulation(leaf)
            self.backpropagate(leaf, simulation_result)
        print("Best child", self.get_best_child(root))

        return self.get_move_made(self.get_best_child(root).get_state(),root.get_state())

In [6]:
class TicTacToe:
    def __init__(self, size, human_player = 'X', machine_player = 'O'):
        self.size = size
        self.board = Board(size)
        self.human = human_player
        self.machine = machine_player
        self.machine_player = MachinePlayer(self.machine, 80)
    
    def play_game(self):
        print(self.board.print_board())
        while not self.board.game_is_over():
            if self.board.turn == self.human:
                self.human_move()
            else:
                self.machine_move()
        print("Game over")
        if self.board.get_winner() == 0:
            print("It's a draw")
        elif self.board.get_winner() == self.human:
            print("You won against the machine! Congrats ", self.board.get_winner())
        else:
            print("Machine takeover! Congrats ", self.board.get_winner())

    def human_move(self):
        chosen_positions = input("Enter the position i,j you want to play in, separated by a comma (enter quit if you want to stop playing): ")
        row, col = map(int, chosen_positions.split(','))
        print("Your move", row, col)
        if self.board.make_move(row, col):
            print(self.board.print_board())
        else:
            print("Invalid move, try again")
            self.human_move()
    
    def machine_move(self):
        move = self.machine_player.get_best_move(self.board)
        print("Machine move: ", move)
        self.board.make_move(move[0], move[1])
        print(self.board.print_board())
        

In [7]:
game = TicTacToe(3)
game.play_game()

0 0 0 
0 0 0 
0 0 0 



ValueError: invalid literal for int() with base 10: ''